In [1]:
import numpy as np

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
import torch.optim as optim

In [5]:
from torchvision import datasets,transforms

In [6]:
import random

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,30)
        self.fc2=nn.Linear(30,10)
    def forward(self,x):
        l1=self.fc1(x)
        al1=torch.sigmoid(l1)
        l2=self.fc2(al1)
        al2=torch.sigmoid(l2)
        return al2
    

In [19]:
def train(model,use_cuda,train_loader,optimizer,epoch):
    model.train()
    for batchid,(data,target) in enumerate(train_loader):
        y_onehot = torch.zeros([target.shape[0], 10])  # Zero vector of shape [64, 10]
        y_onehot[range(target.shape[0]), target] = 1

        data = data.view([data.shape[0], 784])
        if use_cuda:
            data,y_onehot=data.cuda(),y_onehot.cuda()
        optimizer.zero_grad()
        output=model(data)
        loss=torch.mean((output-y_onehot)**2)
        loss.backward()
        optimizer.step()
        if batchid % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batchid * len(data), len(train_loader.dataset),
            100. * batchid / len(train_loader), loss.item()))

In [21]:
def test(model,use_cuda,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data=data.reshape([data.shape[0],784])
            y_onehot=torch.zeros([target.shape[0],10])
            y_onehot[range(target.shape[0]),target]=1
            if use_cuda:
                data,y_onehot=data.cuda(),y_onehot.cuda()
            output=model(data)
            test_loss+=torch.sum((output-y_onehot)**2)
            pred=output.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item() 
        test_loss/=len(test_loader.dataset)
        print(test_loss,100*correct/len(test_loader.dataset))

In [10]:
def seed(seed_value):
    torch.cuda.manual_seed_all(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    torch.backends.cudnn.benchmark=False
    torch.backends.cudnn.deterministic=True

In [15]:
def main():
    use_cuda=False
    seed(0)
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ])
    data1=datasets.MNIST('../data',download=True,train=True,transform=transform)
    data2=datasets.MNIST('../data',train=False,transform=transform)
    train_loader=torch.utils.data.DataLoader(data1,num_workers=6,batch_size=64,shuffle=True)
    test_loader=torch.utils.data.DataLoader(data2,num_workers=6,batch_size=1000,shuffle=False)
    model=Net()
    if use_cuda:
        model=model.cuda()
    optimizer=optim.SGD(model.parameters(),lr=10)
    for epoch in range(1,11):
        train(model,use_cuda,train_loader,optimizer,epoch)
        test(model,use_cuda,test_loader)
    torch.save(model.state_dict(),"mnist_cnn.pt")
    model.load_state_dict(torch.load("mnist_cnn.pt"))

In [22]:
if __name__=="__main__":
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.284883
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.033432
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.021424
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.014365
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.013061
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.012427
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.015567
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.019503
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.025899
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.012034
tensor(0.1259) 92.86
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.015895
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.007555
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.015096
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.018973
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.015171
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.012200
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.014843
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.014440
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.008070
Train Epoch: 2 [57600/60000 (96%)]	Loss: 

In [26]:
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):  # Constructor

        super(Net, self).__init__()

        # 2 Layers
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)

        # xavier_initialization

    def forward(self, x):
        # x - shape = [64, 784]
        # Layer 1
        l1 = self.fc1(x)  # l1 - shape = [64, 100]

        # Activation 1
        l1_a1 = torch.sigmoid(l1)  # l1_a1 - shape = [64, 100]

        # Layer 2
        l2 = self.fc2(l1_a1)  # l2 - shape = [64, 10]

        # Activation 2
        l2_a2 = torch.sigmoid(l2)  # l2_a2 - shape = [64, 10]
        
        return l2_a2


def train(model, use_cuda, train_loader, optimizer, epoch):

    model.train()  # Tell the model to prepare for training
    
    for batch_idx, (data, target) in enumerate(train_loader):  # Get the batch

        # Converting the target to one-hot-encoding from categorical encoding
        # Converting the data to [batch_size, 784] from [batch_size, 1, 28, 28]

        y_onehot = torch.zeros([target.shape[0], 10])  # Zero vector of shape [64, 10]
        y_onehot[range(target.shape[0]), target] = 1

        data = data.view([data.shape[0], 784])

        if use_cuda:
            data, y_onehot = data.cuda(), y_onehot.cuda()  # Sending the data to the GPU

        optimizer.zero_grad()  # Setting the cumulative gradients to 0
        output = model(data)  # Forward pass through the model
        loss = torch.mean((output - y_onehot)**2)  # Calculating the loss
        loss.backward()  # Calculating the gradients of the model. Note that the model has not yet been updated.
        optimizer.step()  # Updating the model parameters. Note that this does not remove the stored gradients!

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, use_cuda, test_loader):

    model.eval()  # Tell the model to prepare for testing or evaluation

    test_loss = 0
    correct = 0

    with torch.no_grad():  # Tell the model that gradients need not be calculated
        for data, target in test_loader:  # Get the batch

            # Converting the target to one-hot-encoding from categorical encoding
            # Converting the data to [batch_size, 784] from [batch_size, 1, 28, 28]

            y_onehot = torch.zeros([target.shape[0], 10])
            y_onehot[range(target.shape[0]), target] = 1
            data = data.view([data.shape[0], 784])

            if use_cuda:
                data, target, y_onehot = data.cuda(), target.cuda(), y_onehot.cuda()  # Sending the data to the GPU

            # argmax([0.1, 0.2, 0.9, 0.4]) => 2
            # output - shape = [1000, 10], argmax(dim=1) => [1000]
            output = model(data)  # Forward pass
            test_loss += torch.sum((output - y_onehot)**2)  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the maximum output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Get total number of correct samples

    test_loss /= len(test_loader.dataset)  # Accuracy = Total Correct / Total Samples

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def seed(seed_value):

    # This removes randomness, makes everything deterministic

    torch.cuda.manual_seed_all(seed_value)  # if you are using multi-GPU.
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def main():

    use_cuda = False  # Set it to False if you are using a CPU
    # Colab And Kaggle

    seed(0)  # Used to fix randomness in the code! Very important!

    # Convert the dataset to tensor and subtract the mean and divide by standard
    # deviation. Why? So that neurons don't saturate!
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # ((mean), (std))    (x - mean)/std
    ])
    # [a1, a2, a3] / [b1, b2, b3] = [a1/b1, a2/b2, a3/b3]

    # Wx + b =~ b

    dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)  # Get the train dataset
    dataset2 = datasets.MNIST('../data', train=False, transform=transform)  # Get the test dataset

    # Get the train data-loader
    train_loader = torch.utils.data.DataLoader(dataset1, num_workers=6, shuffle=True, batch_size=64)
    # Get the test data-loader
    test_loader = torch.utils.data.DataLoader(dataset2, num_workers=6, shuffle=False, batch_size=1000)

    model = Net()  # Get the model

    if use_cuda:
        model = model.cuda()  # Put the model weights on GPU

    optimizer = optim.SGD(model.parameters(), lr=10)  # Choose the optimizer and the set the learning rate

    for epoch in range(1, 10 + 1):
        train(model, use_cuda, train_loader, optimizer, epoch)  # Train the network
        test(model, use_cuda, test_loader)  # Test the network

    torch.save(model.state_dict(), "mnist.pt")

    model.load_state_dict(torch.load('mnist.pt'))

    # Loading a saved model - model.load_state_dict(torch.load('mnist_cnn.pt'))


if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.276752
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.025827
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.012155
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.020065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.011308
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.012245
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.016315
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.020948
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.011743
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.007604

Test set: Average loss: 0.1007, Accuracy: 9388/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.008053
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.015624
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.008314
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.014319
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.010612
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.002161
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.013032
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.007575
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.013966
T

In [3]:
import fasttext


ModuleNotFoundError: No module named 'fasttext'